# Markov graph clustering

### Importing libraries

In [41]:
# Numpy
import numpy as np
from numpy import concatenate, array
from numpy.random import randn

# Matplotlib
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
%matplotlib inline
#mpl.rc('figure', figsize=(10, 8))

# DBscan from sklearn
from sklearn import cluster, datasets
from sklearn.cluster import DBSCAN
from sklearn import metrics
print 'All libraries loaded.'

All libraries loaded.


### Reading from the AT&T dataset

In [42]:
# Configurations for parameters
R = 2.0
pruning_threshold = 0.002
matrix_power_parameter = 2.0
convergence_tolerance = 0.0000000001


def getAdjacencyMatrixFromDataset(filePath):
    # Read from the file
    lines = np.loadtxt(filePath, delimiter=' ', dtype=int)

    # Initialize the adjacency matrix A
    numOfNodes = int(np.amax(lines))
    A = np.zeros((numOfNodes, numOfNodes))
    
    for line in lines:
        A[line[0]-1][line[1]-1] = 1
        A[line[1]-1][line[0]-1] = 1
    
    # Add self-loops
    for i in range(0, numOfNodes):
        A[i][i] = 1
    
    return A

def hasConverged(X, Y):
    # Check if the matrices are equal with a tolerance of 0.001
    return np.allclose(X, Y, atol=convergence_tolerance)

def prune(X):
    for index in np.ndindex(X.shape):
        value = X[index]
        if value <= pruning_threshold:
            X[index] = 0
    return X

A = getAdjacencyMatrixFromDataset("dataset/small.txt")
# A = getAdjacencyMatrixFromDataset("dataset/attweb_net.txt")

In [43]:
# Precision value to display in the matrix
np.set_printoptions(precision=5, suppress=True)

M = np.array(A)
print '\n', M

# Normalize
M = M / M.sum(axis=0)

# Keep an original copy
originalM = np.array(M)
print '\nAfter normalization\n', M

count = 1
while True:
    print '\n\n======= Iteration', count, '========'
    previousM = np.array(M)
    
    # Expand
    M = np.dot(M, originalM)
    print '\nExpanding\n', M
    
    # Inflate
    M = np.power(M, R)
    print '\nInflating\n', M
    
    # Re-normalize
    M = M / M.sum(axis=0)
    print '\nRenormalizing\n', M

    # Pruning
    M = prune(M)
    print '\nPruning\n', M

    # Check for convergence
    if hasConverged(M, previousM):
        break

    count = count + 1

print count


[[ 1.  1.  1.  1.  0.  0.  0.]
 [ 1.  1.  0.  1.  0.  0.  0.]
 [ 1.  0.  1.  0.  1.  1.  1.]
 [ 1.  1.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.]]

After normalization
[[ 0.25     0.33333  0.2      0.33333  0.       0.       0.     ]
 [ 0.25     0.33333  0.       0.33333  0.       0.       0.     ]
 [ 0.25     0.       0.2      0.       0.5      0.5      0.5    ]
 [ 0.25     0.33333  0.       0.33333  0.       0.       0.     ]
 [ 0.       0.       0.2      0.       0.5      0.       0.     ]
 [ 0.       0.       0.2      0.       0.       0.5      0.     ]
 [ 0.       0.       0.2      0.       0.       0.       0.5    ]]


======= Iteration 1 ========

Expanding
[[ 0.27917  0.30556  0.09     0.30556  0.1      0.1      0.1    ]
 [ 0.22917  0.30556  0.05     0.30556  0.       0.       0.     ]
 [ 0.1125   0.08333  0.39     0.08333  0.35     0.35     0.35   ]
 [ 0.22917  0.30556  0.05     0.30556  0.       0.       